<a href="https://colab.research.google.com/github/Fantompp/STA130-HW5/blob/main/STA130_HW7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CHATGPT Chatlogs:**
https://chatgpt.com/share/67400c1b-56d4-800e-b585-20bb368748ed

4. Explain the apparent contradiction between the factual statements regarding the fit below that "the model only explains 17.6% of the variability in the data" while at the same time "many of the coefficients are larger than 10 while having strong or very strong evidence against the null hypothesis of 'no effect'"

The intuition behind this is a difference between the total variability there is in the data, and the coefficients/evidence.
In particular, we've identified strong evidence that generation of pokemon and special defense has some rather large positive effect on the HP stat. So generally speaking, post-gen 1 pokemon have higher HP, and pokemon with higher spdef have higher HP.

This explains some of the variability in the original data, however, there are many other factors which impact the HP stat of a pokemon. If this model were to explain 100% of the variability, it would mean we could predict the HP of a pokemon just from its generation and spdef, which is clearly not true. For example, Pikachu and Machoke are both gen 1 pokemon with a special defense of 60, but Pikachu has a lowly 45 HP, while Machoke has a whopping 80.

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/KeithGalli/pandas/master/pokemon_data.csv"
# fail https://github.com/KeithGalli/pandas/blob/master/pokemon_data.csv
pokeaman = pd.read_csv(url)
pokeaman
import statsmodels.formula.api as smf

model1_spec = smf.ols(formula='HP ~ Q("Sp. Def") + C(Generation)', data=pokeaman)
model2_spec = smf.ols(formula='HP ~ Q("Sp. Def") + C(Generation) + Q("Sp. Def"):C(Generation)', data=pokeaman)
model2_spec = smf.ols(formula='HP ~ Q("Sp. Def") * C(Generation)', data=pokeaman)

model2_fit = model2_spec.fit()
model2_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     HP   R-squared:                       0.176
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     15.27
Date:                Fri, 22 Nov 2024   Prob (F-statistic):           3.50e-27
Time:                        00:43:14   Log-Likelihood:                -3649.4
No. Observations:                 800   AIC:                             7323.
Df Residuals:                     788   BIC:                             7379.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                          26.8971      5.246      5.127      0.000      16.599      37.195
C(Generation)[T.2]                 20.0449      7.821      2.563      0.011       4.692      35.398
C(Generation)[T.3]                 21.3662      6.998      3.053      0.002       7.629      35.103
C(Generation)[T.4]                 31.9575      8.235      3.881      0.000      15.793      48.122
C(Generation)[T.5]                  9.4926      7.883      1.204      0.229      -5.982      24.968
C(Generation)[T.6]                 22.2693      8.709      2.557      0.011       5.173      39.366
Q("Sp. Def")                        0.5634      0.071      7.906      0.000       0.423       0.703
Q("Sp. Def"):C(Generation)[T.2]    -0.2350      0.101     -2.316      0.021      -0.434      -0.036
Q("Sp. Def"):C(Generation)[T.3]    -0.3067      0.093     -3.300      0.001      -0.489      -0.124
Q("Sp. Def"):C(Generation)[T.4]    -0.3790      0.105     -3.600      0.000      -0.586      -0.172
Q("Sp. Def"):C(Generation)[T.5]    -0.0484      0.108     -0.447      0.655      -0.261       0.164
Q("Sp. Def"):C(Generation)[T.6]    -0.3083      0.112     -2.756      0.006      -0.528      -0.089
==============================================================================
Omnibus:                      337.229   Durbin-Watson:                   1.505
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2871.522
Skew:                           1.684   Prob(JB):                         0.00
Kurtosis:                      11.649   Cond. No.                     1.40e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

7. Discuss with a ChatBot the rationale and principles by which model5_linear_form is extended and developed from model3_fit and model4_fit; model6_linear_form is extended and developed from model5_linear_form; and model7_linear_form is extended and developed from model6_linear_form; then, explain this breifly and consisely in your own words

After a nice chat with the Chat, here's what I found:

Model 5 basically slaps a whole bunch of variables into the model, specifically legendary status and the other stats. My pokemon intuition says that all of these should have some effect on the HP, but they mostly serve to explain the same thing (legendary mons have high stats all around, and pokemon with high stats tend to have other high stats, up to a certain point). It also adds type, for fun. (I notice it treats first and second type differently, which is interesting).

Model 6 refines it removing many of the unimportant variables, and prunes the types only to the relevant types (supposedly, though it's more likely just random types for the sake of example).

Model 7 adds some interactions to try to better see how the different stats affect HP together (which should help remove some of the colinearity and better capture behavior).

Model 7+ centers and scales the values, essentially normalizing their value and deviation to try to remove colinearity. This probably solves some of what I mentioned about high stat pokemon having other high stats.

9. Work with a ChatBot to understand the meaning of the illustration below; and, explain this in your own words

Chat was a little vague about this one, but as far as I understand, we're doing a bunch of testing to make sure our model is actually robust. Since we have multiple generations of data, rather than the typical train test where we just reserve 20% of the data, we can actually instead just train our data on all but one of the generations, and see how well it predicts the one left out.

And in doing so, we show how well our data will generalize to different generations (assuming new generations will roughly be like the ones we currently have).

The two examples we do to illustrate this are: Using gen 1 to predict the future gens, and using gen not-6 to predict gen 6.

We also do this to see the difference between model 6 and 7. This lets us make sure that these models are overfit to their training data, which becomes a natural concern as the models become more complex, particularly with model 7.